# PART1 基礎分析をしよう

## おまじない

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install japanize-matplotlib
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import japanize_matplotlib
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from sklearn.linear_model import LinearRegression as LR

Mounted at /content/drive
     |████████████████████████████████| 4.1 MB 5.4 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=fed4f7b0edcd39eff87b0ad36f9628770efb7c6ce5f0eb46d469472d679e7d02
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## データの読み込み
- train.csvとtest.csvとsubmit_sample.csvの３つのファイルを読み込みましょう
- それぞれを読み込んだものは変数はtrain, test, sampleに代入しましょう

In [3]:
train = pd.read_csv('/content/drive/MyDrive/practice/practice_data/train2.csv')
test = pd.read_csv('/content/drive/MyDrive/practice/practice_data/test2.csv')
sample = pd.read_csv('/content/drive/MyDrive/practice/practice_data/submit_sample2.csv', header = None)

## trainの先頭行を見てみましょう
- 各カラムの説明はDeepAnalyticsの[データダウンロードページ](https://deepanalytics.jp/compe/1/download)に記載されています

In [4]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,39,blue-collar,married,secondary,no,1756,yes,no,cellular,3,apr,939,1,-1,0,unknown,1
1,2,51,entrepreneur,married,primary,no,1443,no,no,cellular,18,feb,172,10,-1,0,unknown,1
2,3,36,management,single,tertiary,no,436,no,no,cellular,13,apr,567,1,595,2,failure,1
3,4,63,retired,married,secondary,no,474,no,no,cellular,25,jan,423,1,-1,0,unknown,1
4,5,31,management,single,tertiary,no,354,no,no,cellular,30,apr,502,1,9,2,success,1


## trainの行数と列数を見てみましょう

In [5]:
train.shape

(27128, 18)

## trainの基礎統計量を見てみましょう

In [7]:
train.describe()

,id,age,balance,day,duration,campaign,pdays,previous,y
count,27128.000000,27128.000000,27128.000000,27128.000000,27128.000000,27128.000000,27128.000000,27128.000000,27128.000000
mean,13564.500000,40.951010,1355.800870,15.806215,260.711295,2.751769,40.528052,0.579733,0.117001
std,7831.323388,10.608542,3003.305272,8.337904,260.091727,3.126594,100.382462,2.503653,0.321427
min,1.000000,18.000000,-6847.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6782.750000,33.000000,72.000000,8.000000,104.000000,1.000000,-1.000000,0.000000,0.000000
50%,13564.500000,39.000000,449.000000,16.000000,182.000000,2.000000,-1.000000,0.000000,0.000000
75%,20346.250000,48.000000,1428.000000,21.000000,323.000000,3.000000,-1.000000,0.000000,0.000000
max,27128.000000,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000,1.000000


## testの基礎統計量も見てみましょう

In [8]:
test.describe()

,id,age,balance,day,duration,campaign,pdays,previous
count,18083.000000,18083.000000,18083.000000,18083.000000,18083.000000,18083.000000,18083.000000,18083.000000
mean,9042.000000,40.914008,1371.980092,15.806725,254.340264,2.781950,39.702428,0.581209
std,5220.256794,10.634331,3105.985293,8.299509,253.591981,3.054651,99.747634,1.965265
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,4521.500000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,9042.000000,39.000000,447.000000,16.000000,178.000000,2.000000,-1.000000,0.000000
75%,13562.500000,48.000000,1427.000000,21.000000,314.000000,3.000000,-1.000000,0.000000
max,18083.000000,95.000000,98417.000000,31.000000,3881.000000,50.000000,854.000000,58.000000


## trainとtestに欠損がないか見てみましょう

In [9]:
train.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [10]:
test.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64

## trainでyが1の人数はどれくらいか見てみましょう

In [11]:
train['y'].value_counts()

0    23954
1     3174
Name: y, dtype: int64

## maritalとyのクロス集計をしてみましょう
- pd.crosstab関数を使います
- pd.crosstab(X["A"], X["B"])と書いた場合、Aが縦列、Bが横列となります
- 更にオプションとしてmargins=Trueを書くと、総計値のカラムまで作成されるので便利です

In [13]:
pd.crosstab(train['marital'], train['y'], margins = True)

y,0,1,All
marital,,,
divorced,2691,364,3055
married,14744,1667,16411
single,6519,1143,7662
All,23954,3174,27128


## ageをビニングしてみましょう
- ageは数値データなので、クロス集計をする為にはビニングが必要です
- ビニングとは数値データを例えば、グループ①（0より大きい、10以下）、グループ②（10より大きい、20以下）…のように集約することを言います
- ビニングはpd.cut関数を使います
- オプションには、①ビニングしたいデータ、②どう区切るのか？（例えば[0,10,20]）を書きます
- まずtrain["age"]の基本統計量を確認しましょう
- その後、ビニングした結果を変数age_biningに代入しましょう

In [14]:
train['age'].describe()

count    27128.000000
mean        40.951010
std         10.608542
min         18.000000
25%         33.000000
50%         39.000000
75%         48.000000
max         95.000000
Name: age, dtype: float64

In [16]:
age_bining = pd.cut(train['age'],[0, 20, 30, 40, 50, 60, 100])

## age_biningとyを使ってクロス集計をしてみましょう

In [17]:
pd.crosstab(age_bining, train['y'], margins = True)

y,0,1,All
age,,,
"(0, 20]",39,21,60
"(20, 30]",3486,655,4141
"(30, 40]",9488,1111,10599
"(40, 50]",6117,602,6719
"(50, 60]",4416,488,4904
"(60, 100]",408,297,705
All,23954,3174,27128


# 実習

1.testの行数とカラム数を確かめてみよう

In [18]:
test.shape

(18083, 17)

2.trainのデータには数値データが何カラムあり、質的データが何カラムあるか調べてみよう

In [23]:
train.info

<bound method DataFrame.info of           id  age           job  marital  ... pdays previous  poutcome  y
0          1   39   blue-collar  married  ...    -1        0   unknown  1
1          2   51  entrepreneur  married  ...    -1        0   unknown  1
2          3   36    management   single  ...   595        2   failure  1
3          4   63       retired  married  ...    -1        0   unknown  1
4          5   31    management   single  ...     9        2   success  1
...      ...  ...           ...      ...  ...   ...      ...       ... ..
27123  27124   42   blue-collar  married  ...   269        2     other  0
27124  27125   34      services   single  ...    -1        0   unknown  0
27125  27126   36   blue-collar   single  ...    -1        0   unknown  0
27126  27127   33     housemaid  married  ...    -1        0   unknown  0
27127  27128   29      services   single  ...   355        2   failure  0

[27128 rows x 18 columns]>

3.poutcomeとyのクロス集計をし、変数poutに代入しよう
- margins=Trueはつけましょう

In [21]:
pout = pd.crosstab(train['poutcome'], train['y'], margins = True)

4.trainのpoutcomeの各値毎に1となる割合を計算し、poutに新たなカラムrateを追加しよう
- poutcomeの各値の合計値に対して1となる割を求めることになるので、pout[1] / pout["All"] で求めることができます

In [25]:
pout['rate'] = pout[1] / pout['All']

5.trainのdurationをビニングして、変数duration_biningを作りましょう
- train["duration"]の基礎統計量を確認しておきましょう
- 区切り方は[-1,100,200,300,400,500,600,700,800,900,1000,5000]としましょう

In [28]:
duration_bining = pd.cut(train['duration'], [-1, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 5000])

6.duration_biningとyを使って、クロス集計をし、変数duraに代入しよう
- margins=Trueはつけましょう

In [33]:
dura = pd.crosstab(duration_bining, train['y'], margins = True)

7.duraの各値毎に1となる割合を計算し、duraに新たなカラムrateを追加しよう

In [32]:
dura['rate'] = dura[1] / dura['All']